In [2]:
# Creating a new ABM using a network structure

In [1]:
# pip install if running from ucloud either you can install to conda env
pip install mesa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.7 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.1 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 443.3 kB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 341.8 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
# loading packages
import random
import numpy as np

import pandas as pd
from collections import Counter
import itertools
from datetime import datetime

import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import StagedActivation, BaseScheduler
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.datacollection import DataCollector

# Creating the ABM

## Custom functions
Creating some custom functions to run later inside the MESA objects/modules

In [2]:
# Simulating agents parameters

# generating pisa mean scores
def generate_pisa_score():
    score = np.random.normal(500, 95)
    return score

# generating cps levels
def generate_cps_level(x):
   # if x is below 340 then cps level is 0
   # if x is between 340 and 440 then cps level is 1
    # if x is between 440 and 540 then cps level is 2
    # if x is between 540 and 640 then cps level is 3
    # if x is above 640 then cps level is 4
    
    if x < 340:
        cps_level = 0
    elif x >= 340 and x < 440:
        cps_level = 1
    elif x >= 440 and x < 540:
        cps_level = 2
    elif x >= 540 and x < 640:
        cps_level = 3
    elif x >= 640:
        cps_level = 4
    return cps_level


# Generating a function which creates a task work level which is a random number between 0 and 1 and then multiplied by x and add uncertainty around this estimate of 0.77

# finding agents with the same role
def get_agents_with_same_role(team_current_roles, team_id, agent_id):
    role = team_current_roles[team_id][agent_id]
    agents_with_same_role = [agent for agent, agent_role in team_current_roles[team_id].items() if agent_role == role]
    return agents_with_same_role

# unnest list in list function
def unnest_list(nested_list):
    result = []
    for item in nested_list:
        if isinstance(item, list):
            result.extend(unnest_list(item))
        else:
            result.append(item)
    return result

# remove duplicates
def remove_duplicates(original_list):
    result_list = []
    for item in original_list:
        if item not in result_list:
            result_list.append(item)
    return result_list

## Custom Staged activation
Modidying the MESA module stagedactivation which is a scheduler to include a nested loop
They Moduler takes two new attributes which is num_rounds and self.p_round

num_rounds = a self determined variable put into the model, which the determines the number of rounds each problem is worked on i.e. how many times the nested loop of stages 2 to 6 are repeated.

p_round = the current problem_round, this variable increases from 0 each time the loop is run through. It is used later on to keep track of the current round in the loop

In [3]:
from mesa.time import StagedActivation

class CustomStagedActivation(StagedActivation):
    def __init__(self, model, stage_list, num_rounds, shuffle=False, shuffle_between_stages=False):
        super().__init__(model, stage_list, shuffle, shuffle_between_stages)
        self.num_rounds = num_rounds
        self.p_round = 0

    def step(self):
        """Executes all the stages for all agents, including a loop alternating between stages two and three."""
        # To be able to remove and/or add agents during stepping
        # it's necessary to cast the keys view to a list.
        agent_keys = list(self._agents.keys())
        if self.shuffle:
            self.model.random.shuffle(agent_keys)
        for stage in self.stage_list:
            self.p_round = 0
            if stage == "stage_one":
                for agent_key in agent_keys:
                    if agent_key in self._agents:
                        getattr(self._agents[agent_key], 'stage_one')()
            elif stage == "stage_two":
                for _ in range(self.num_rounds):  # Loop for stage two and three (num_rounds times)
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], 'stage_two')()  # Run stage
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], "stage_three")()  # Run stage
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], "stage_four")()
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], "stage_five")()
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], "stage_six")()
                    
                    self.p_round += 1
            elif stage == "stage_seven":
                for agent_key in agent_keys:
                    if agent_key in self._agents:
                        getattr(self._agents[agent_key], "stage_seven")()  # Run stage
            # We recompute the keys because some agents might have been removed
            # in the previous loop.
            agent_keys = list(self._agents.keys())
            if self.shuffle_between_stages:
                self.model.random.shuffle(agent_keys)
            self.time += self.stage_time

        self.steps += 1


## TeamAgent

In [4]:
class TeamAgent(Agent):
    """ An agent with X."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.pisa_score = generate_pisa_score()
        self.cps_level = generate_cps_level(self.pisa_score)
        
        self.team_members = []
        self.team_id = None
        self.team_cps = []
        
        self.current_role = None
        self.correct_role = None
        self.role_disagreement = 0  # can be 0 or 1
        self.is_role_correct = 0
        self.agents_with_same_role = []

        self.solution = 0

        self.common_ground = 0

        # variables for data collection
        self.team_size = 0
        self.total_conflicts = 0
        self.total_conflicts_resolved = 0
        self.total_common_ground = 0
        self.all_best_solutions = []
        self.team_best_solutions_mean = []
        self.problem_complexity = 0
        self.p_rounds = self.model.rounds
        self.problem_complexity_distribution = self.model.problem_complexity_distribution
        

    # Preparing agents into teams
    def reset_team(self):
        self.team_members = []
        self.team_id = None

        self.total_common_ground = 0
        self.total_conflicts = 0
        self.total_conflicts_resolved = 0
        


    ## Joining teams
    def join_teams3(self):
        self.team_id = self.model.agent_team_dict.get(self.unique_id)

        #other_agent_ids = [agent_id for agent_id, t_id in self.model.agent_team_dict.items() if t_id == self.team_id and agent_id != self.unique_id]
        #self.team_members = other_agent_ids + [self.unique_id]
        members = [agent_id for agent_id, t_id in self.model.agent_team_dict.items() if t_id == self.team_id]
        self.team_members = members

        self.team_size = len(self.team_members)

        print("agent_id:", self.unique_id, "Team_id:", self.team_id, "Team_members:", self.team_members)

    ## Creating a correct role
    def create_roles(self):

        x = list( range(1, len(self.model.teams[self.team_id])) )
        self.model.teams[self.team_id].append()
    

        self.model.teams[self.team_id]

    def get_correct_role(self):
        
        for i in self.team_members:
            self.correct_role = self.team_members.index(self.unique_id)

        print("Your correct role is:", self.correct_role)
    

    def grab_team_cps(self):
        self.team_cps = []

        for i in self.team_members:
            self.team_cps.append( self.model.schedule.agents[i].cps_level )

        print("This is your own cps:", self.cps_level)
        print("This is the cps of whole team:", self.team_cps)
        
    # Defining the problem loop where evaluation takes place

    def get_start_role(self):
        alist = list(range(len(self.team_members)))
        alist.remove(self.correct_role)
        other_roles = alist


        if 3 in self.team_cps or 4 in self.team_cps:
            self.current_role = self.correct_role # should change this to just be the prob of choice
            print("First round/correct_role_assigned.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        else:
            self.current_role = np.random.choice(other_roles)
            print("First round/random_role_assigned.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        
        print("Your starting role is:", self.current_role)
        print("Your correct role is:", self.correct_role)
        

    def get_current_role(self):
        # Simulate a current_role for each agent from 0 to length of team_members the curren_role should have a 80% probability of being correct_role and a 20% probability of being a random role
        alist = list(range(len(self.team_members))).copy()
        alist.remove(self.correct_role)
        other_roles = alist

        alist2 = list(range(len(self.team_members))).copy()
        print("length of self team mebers", len(self.team_members) )
        print("alist2:", alist2, "current_role:", self.current_role, "Agent:", self.unique_id, "team:", self.team_id, "alist:", alist)
        alist2.remove(self.current_role)
        other_roles2 = alist2

        # propability list where the correct role has a 80% probability and the other roles have a 20% probability
        ## if all cps are below 3 then the probability of the role which was taken in the previous round is 50% and the probability of all other roles is 50% / number of other roles
        prob_role_disagreement_true = [0.5] + [0.5 / len(other_roles2)] * len(other_roles2)
        # if all 
        prob_role_disagreement_false = [0.75] + [0.25 / len(other_roles2)] * len(other_roles2)
        p_role_high = [0.95] + [0.05 / len(other_roles)] * len(other_roles)
        
        

        if all(self.team_cps) < 3:
            if self.role_disagreement == 1:
                self.current_role = np.random.choice([self.current_role] + other_roles2, p=prob_role_disagreement_true)
                print("Role Disagreement." ,"agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
            else:
                self.current_role = np.random.choice([self.current_role] + other_roles2, p=prob_role_disagreement_false)
                print("No Role Disagreement." ,"agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        elif 3 in self.team_cps and all(self.team_cps) < 4 and self.cps_level == 3:
            self.current_role = np.random.choice([self.correct_role] + other_roles, p=p_role_high)
            print("Level 3 and no 4s in team.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        elif 3 in self.team_cps and all(self.team_cps) < 4 and self.cps_level < 3:
            self.current_role = np.random.choice([self.correct_role] + other_roles, p=prob_role_disagreement_false)
            print("Level 3 and no 4s in team.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        elif 4 in self.team_cps:
            self.current_role = np.random.choice([self.correct_role] + other_roles, p=p_role_high)
            print("Level 4 in team.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
    
        
        print("Your current role is:", self.current_role)
        print("Your correct role is:", self.correct_role)

    def getting_roles(self):
            # creating the roles
        if self.model.schedule.p_round == 0:
            self.get_start_role()
        else:
            self.get_current_role()
        
        if self.team_id in self.model.team_current_roles:
            if self.unique_id in self.model.team_current_roles[self.team_id]:
                self.model.team_current_roles[self.team_id][self.unique_id].append(self.current_role)
            else:
                self.model.team_current_roles[self.team_id][self.unique_id] = [self.current_role]
        else:
            self.model.team_current_roles[self.team_id] = {self.unique_id: [self.current_role]}
            
        print("This is the current roles of my team:", self.model.team_current_roles[self.team_id])

    # finding out if theres been 2 people in the same role
    def find_role_disagreement(self):

        for i in self.model.team_current_roles[self.team_id]:
            if self.model.team_current_roles[self.team_id].count(i) > 1:
                self.role_disagreement = 1
            else:
                self.role_disagreement = 0
        
        
        # find who is disagreeing
    def find_agents_with_same_role(self):
        
        agents_with_same_role_list = [agent for agent, agent_role in self.model.team_current_roles[self.team_id].items() if agent_role == self.current_role]
        
        self.agents_with_same_role = agents_with_same_role_list

        print("this is the current roles of my team:", self.model.team_current_roles[self.team_id])
        print("this is the agents with same role", self.agents_with_same_role)

    
    
       
    def establish_common_ground(self):
        # common ground might be = 1 with a default prob of 50% however if there is a cps_lvl 2 in the team_members then add the weighted probability of cps_lvl

        prob = 0.5
        for i in self.team_cps:
            if i > 1:
                if i == 2:
                    prob += ( 0.5/len(self.team_cps) ) * 0.7 # The combination of "to the power of" and 1.05 creates a good balanced probability of synergy based on the incremental charecteristics
                elif i == 3:
                    prob += ( 0.5/len(self.team_cps) ) * 0.85
                elif i == 4:
                    prob += ( 0.5/len(self.team_cps) ) * 1

        
        # generate a common ground 0 or 1 for each agent
        common_ground = np.random.binomial(1, prob)

        # append this to a dictionary with team as key and common_ground as value if team is not in the dictionary already
    
        if self.team_id not in list( self.model.team_common_ground.keys() ):
            self.model.team_common_ground[self.team_id] = [common_ground]

        self.common_ground = common_ground

        if self.model.team_common_ground[self.team_id] == [1]:
            self.total_common_ground += 1
        
        print("This is the common ground for", self.team_id, " :", self.model.team_common_ground[self.team_id])
        

    # this loop creates the problem, the roles, common ground etc.
    def problem_loop(self):

        # create a problem_solving round counter
        self.problem_complexity = self.model.problem_complexity

        # If the task fits level
        if self.cps_level >= self.model.problem_complexity:
            self.solution = random.uniform(0.9, 1.1) * self.pisa_score # 10% uncertainty around the solution because of the gap in cps level point range
        # If the task is too complex
        elif self.cps_level < self.model.problem_complexity:
            self.solution = 0 # FOR NOW! this will remain 0 I might change this to a prob 
        
        # add the self solution to a model class variable which is indexed by the team_id and agent_id
         
        if self.team_id in self.model.team_solutions:
            if self.unique_id in self.model.team_solutions[self.team_id]:
                self.model.team_solutions[self.team_id][self.unique_id].append(self.solution)
            else:
                self.model.team_solutions[self.team_id][self.unique_id] = [self.solution]
        else:
            self.model.team_solutions[self.team_id] = {self.unique_id: [self.solution]}

        print("agent", self.unique_id, "team", self.team_id, "This is my solution:", self.solution)
        print("This is the team solutions:", self.model.team_solutions)

    # conflict might arise  
    def conflict(self):
        
        if self.team_id not in self.model.team_conflict:
            
            team_current_roles = list( self.model.team_current_roles[self.team_id] )
            
            all_agents_in_disagreement = 0 # creates a count of how many agent who share a role
            for i in team_current_roles:
                if team_current_roles.count(i) > 1:
                    all_agents_in_disagreement += 1
            print("all_agents_in_disagreement", all_agents_in_disagreement)

            if self.common_ground == 1:
                no_common_ground = 1
            else:  
                no_common_ground = 0
            prob = 0.1 + (0.3 * no_common_ground) + ( (0.5/len(self.team_members)) * all_agents_in_disagreement ) 
            print("probability of conflict", prob)

            conflict = np.random.binomial(1, prob)
            

            # append conflict to a team dictionary
        
            self.model.team_conflict[self.team_id] = [conflict]
        else:
            pass

        if self.model.team_conflict[self.team_id] == [1]:
            self.total_conflicts += 1

        print("This is the CONFLICT for", self.team_id, " :", self.model.team_conflict[self.team_id])
            
    def conflict_resolution(self):
        
        if self.team_id not in self.model.team_conflict_resolution:
            if self.model.team_conflict[self.team_id] == [1]:
                
                #if there is a conflict we can try to solve it

                baseline_probability = 0.1
                num_level3_agents = self.team_cps.count(3)
                if num_level3_agents > 0:
                    baseline_probability = 0.6 + ( 0.2/len(self.team_cps) ) * num_level3_agents
                num_level4_agents = self.team_cps.count(4)
                if num_level4_agents > 0:
                    baseline_probability = 0.8 + ( 0.2/len(self.team_cps) ) * num_level4_agents

                print("probability of conflict resolution", baseline_probability)
                conflict_solution = np.random.binomial(1, baseline_probability)
            
                self.model.team_conflict_resolution[self.team_id] = [conflict_solution]

                # Conflict of team is changed to 0 if resolution is 1
                if conflict_solution == 1:
                    self.model.team_conflict[self.team_id] = [0] # CHANGE CONFLICT to 0
                    print("Conflict resolution successful")
            elif self.model.team_conflict[self.team_id] == [0]:
                self.model.team_conflict_resolution[self.team_id] = [0]
        else:
            pass

        if self.model.team_conflict_resolution[self.team_id] == [1]:
            self.total_conflicts_resolved += 1

        print("This is the CONFLICT RESOLUTION for", self.team_id, " :", self.model.team_conflict_resolution[self.team_id])


    def adjust_solution_for_disagreement(self):
        compromise_or_not = np.random.binomial(1, 0.5) # if 1 then compromise if 0 then not
        if len(self.agents_with_same_role) > 1:
            self.role_disagreement = 1 # if there are more than 1 agents with the same role then that agents is in disagreement

            both_solutions = list( [self.model.team_solutions[self.team_id][agent] for agent in self.agents_with_same_role] )
            print("this is both_solutions",both_solutions)
            both_solutions = unnest_list(both_solutions)
            print("this is both_solutions, iterated by unnest_list",both_solutions)
            print("this is both solution for the agents in the same role", both_solutions)
            if compromise_or_not == 1:
                self.model.team_solutions[self.team_id][self.unique_id] = np.mean(both_solutions)
                print("compromise, solution updated", self.model.team_solutions[self.team_id][self.unique_id])
            elif compromise_or_not == 0:
                if self.common_ground == 1:
                    self.model.team_solutions[self.team_id][self.unique_id] = max(both_solutions)
                    print("no compromise but common_ground, solution updated", self.model.team_solutions[self.team_id][self.unique_id])
                elif self.common_ground == 0:
                    random_solution = np.random.choice(both_solutions)
                    self.model.team_solutions[self.team_id][self.unique_id] = random_solution
                    print("no compromise no common_ground, solution updated", self.model.team_solutions[self.team_id][self.unique_id])

    # evaluating and comparing solutions 
    def evaluate(self):
        # print the solutions of all agents in the team

        all_round_team_solutions = list( self.model.team_solutions[self.team_id].values() )
        all_round_team_solutions = unnest_list(all_round_team_solutions) #unnest list from dictionary
        print("all_round_team_solutions before removal of duplicates", all_round_team_solutions)

        all_round_team_solutions = remove_duplicates(all_round_team_solutions) # removing duplicates
        print("all_round_team_solutions after removal", all_round_team_solutions)
       
        if self.model.team_conflict[self.team_id] == [1]:
            team_best_solution = 0
        elif self.model.team_conflict[self.team_id] == [0] and self.common_ground == 1:
            team_best_solution = max(all_round_team_solutions)
        elif self.model.team_conflict[self.team_id] == [0] and self.common_ground == 0:
            team_best_solution = random.choice(all_round_team_solutions)
        print("team_best_solution before appending", team_best_solution)
        

        # evaluate all solutions in the team_solutions dictionary indexed by team_id and store the highest in a model class variable called best_solution indexed by team_id
    
        if self.team_id not in self.model.best_solution:
            self.model.best_solution[self.team_id] = [team_best_solution]
        elif len(self.model.best_solution[self.team_id]) < self.model.schedule.p_round + 1:
            self.model.best_solution[self.team_id].append(team_best_solution)
        else:
            pass

        # print the team id and the best solution in the same line
        print("Common_ground", self.model.team_common_ground[self.team_id], "This is the solution picked for team", self.team_id, "problem round", self.model.schedule.p_round, " :", self.model.best_solution[self.team_id])


    # Resetting variables 
    # before running next generation of solutions and evaluations
    # But most importantly KEEP best_solutions and 
    # might use data collector to collect all team solutions 
    # or i can just create a secondary variable which stores all_team_solutions then append them to the global solutions where all solutions are stored
    def reset_variables(self):

        self.model.team_solutions = {}
        
        self.model.team_common_ground= {}

        self.model.team_current_roles = {}

        self.model.team_conflict = {}

        self.model.team_conflict_resolution = {}

        

    def test_data_collection(self):
        print("This is the test data collection")
        all_best_solutions = list( self.model.best_solution[self.team_id] )
        self.all_best_solutions = all_best_solutions
        
        print("this is all best solutions", all_best_solutions)
        print("this is mean of all best solutions", np.mean(all_best_solutions))
        mean_best_solutions = np.mean(all_best_solutions)
        self.team_best_solutions_mean = mean_best_solutions
        

    # // STAGES //
    def stage_one(self):
        self.reset_team()
        self.join_teams3()
        self.get_correct_role()
        self.grab_team_cps()
    
    
    def stage_two(self):
        print("self.model.schedule.p_round", self.model.schedule.p_round)
        self.establish_common_ground()
        print("DEBUGGIN Agent", self.unique_id, "current role", self.current_role, "correct role", self.correct_role, "common ground", self.common_ground)
        
        self.getting_roles()

    def stage_three(self):
        self.find_agents_with_same_role()
        self.problem_loop()
        self.conflict()    
    
    def stage_four(self):
        self.adjust_solution_for_disagreement()
        self.conflict_resolution()

    def stage_five(self):
        self.evaluate()

    def stage_six(self):
        self.reset_variables()

    def stage_seven(self):
        # here i want to collect data
        self.test_data_collection()
    

    

        
       

# Team Model

In [5]:
# Creating the model
class TeamModel(Model):
    def __init__(self, num_agents, max_team_size, rounds, problem_complexity_distribution):
        self.num_agents = num_agents
        self.rounds = rounds
        self.max_team_size = max_team_size

        self.schedule = CustomStagedActivation(self, stage_list=["stage_one", "stage_two", "stage_three", "stage four", "stage_five", "stage_six", "stage_seven"], num_rounds=self.rounds)
        
        self.teams = {}
        self.agent_team_dict = {}
        self.groups = []

        self.problem_complexity_distribution = problem_complexity_distribution
        self.problem_complexity = 0

        self.team_current_roles = {}
        self.team_common_ground = {}
        self.team_conflict = {}
        self.team_conflict_resolution = {}

        self.team_solutions = {}
        self.best_solution = {}
        
        self.p_round = 0
        
        for i in range(self.num_agents):
            agent = TeamAgent(i, self)
            self.schedule.add(agent)

        self.datacollector = mesa.DataCollector(
            agent_reporters={"p_rounds": "p_rounds", "problem_complexity_distribution": "problem_complexity_distribution", "problem_complexity": "problem_complexity", "team_id": "team_id", "team_cps": "team_cps", "team_size": "team_size", "total_conflicts": "total_conflicts", "total_conflicts_resolved": "total_conflicts_resolved", "total_common_ground": "total_common_ground", "team_best_solutions_mean": "team_best_solutions_mean","all_solutions": "all_best_solutions"}
        )

    # Creating team lists
    def create_teams(self):
        # Creating random teams of 2-5 agents based on the number of agents
        numbers = []
        x = 0
        while len(numbers) < self.num_agents:
            repetitions = random.randint(2, self.max_team_size)
            numbers.extend([x] * repetitions)
            x += 1
        numbers = numbers[:self.num_agents]  # Trim the list to the desired number of agents
        if numbers.count(numbers[self.num_agents - 1]) < 2:
            numbers.remove(numbers[self.num_agents - 1])
            possible_teams = []
            for i in set(numbers):
                if numbers.count(i) < self.max_team_size:
                    possible_teams.append(i)
            numbers.append(random.choice(possible_teams))
        

        random.shuffle(numbers) 

        self.groups = numbers

        # Creating the roles such that
        # each team has 2 to 5 roles specified by 1:n

        all_agents = list( range(0, self.num_agents) )
        self.teams = {
            "agent_id": all_agents,
            "team_id": self.groups
        }

        self.agent_team_dict = {agent_id: team_id for agent_id, team_id in zip(self.teams['agent_id'], self.teams['team_id'])}
        

    # creating a problem which is then used for solving
    def generate_problem(self): 
        self.problem_complexity = np.random.choice([1,2,3], p=self.problem_complexity_distribution)
    
    # A function which checks
    def collect_teams(self):
        print(self.teams)
        print(self.agent_team_dict)

    def reset_variables(self):
        self.teams = {}
        self.agent_team_dict = {}
        self.groups = []

        self.team_current_roles = {}
        self.team_common_ground = {}
        self.team_team_conflict = {}

        self.team_solutions = {}
        self.best_solution = {}

        self.p_round = 0

    def step(self):
        self.reset_variables()
        self.create_teams()
        self.generate_problem()
        self.schedule.step()
        self.collect_teams()
        self.datacollector.collect(self)




# Run the Model

In [9]:
%%capture

model = TeamModel(num_agents=7000, max_team_size = 10, rounds=10, problem_complexity_distribution=[0.333, 0.333, 0.334])

for i in range(30):
    model.step()



In [10]:
merged_dataframe = model.datacollector.get_agent_vars_dataframe()

In [11]:
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Save the merged dataframe to a file with the timestamp in the file name
filename = f"ABM_{timestamp}.csv"
merged_dataframe.to_csv(filename, index=True)